# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# imports


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import tensorflow.keras
from tensorflow.keras import models, layers, optimizers

from	tensorflow.keras.models import	Sequential
from	tensorflow.keras.layers import	Dense,	Activation
from  scipy.stats import zscore

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, r2_score

# Print Config


In [ ]:
#comment to turn off, prints more data.
def show_allData(all):
  if all:
    np.set_printoptions(threshold=np.inf) #numpy show whole array
    pd.set_option('display.max_rows', None) #pandas show whole dataframe
  else:
    np.set_printoptions(threshold=6) #numpy show whole array
    pd.set_option('display.max_rows', 6)

Bold='\033[1m'
UnderLine='\033[4m'
FormatEnd='\033[0m'



#Part A

## Read CSV

In [ ]:
import pandas as pd
wineFile='/content/drive/Shareddrives/Team 8- Neural Network/Assignment-8/RedWineQuality.csv'
df = pd.read_csv(wineFile, header=0)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


### Parse Data


In [ ]:
num_col=len(df.columns)
num_row=len(df)
num_feat=num_col-1

X=df.iloc[:num_row, :num_col-1].values
X=zscore(X)   #normalize


y =	pd.get_dummies(df['quality'])
y=y.values
print(X)



[[-0.52835961  0.96187667 -1.39147228 ...  1.28864292 -0.57920652
  -0.96024611]
 [-0.29854743  1.96744245 -1.39147228 ... -0.7199333   0.1289504
  -0.58477711]
 [-0.29854743  1.29706527 -1.18607043 ... -0.33117661 -0.04808883
  -0.58477711]
 ...
 [-1.1603431  -0.09955388 -0.72391627 ...  0.70550789  0.54204194
   0.54162988]
 [-1.39015528  0.65462046 -0.77526673 ...  1.6773996   0.30598963
  -0.20930812]
 [-1.33270223 -1.21684919  1.02199944 ...  0.51112954  0.01092425
   0.54162988]]


### Split data


In [ ]:
#split test and train data
#X prime and Yprime are the training data after split
X_prime, X_test, y_prime, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
show_allData(False)
X_prime.shape


(1279, 11)

## Kfold and Training



In [ ]:
def kfold_train(model,k,num_epochs,X_prime,y_prime,Kshuffle,Krandom_state):
  """
  Does Kfold and train the model
  Shows the avg accuracy
  returns the model after training
  """
  # k	=	10
  # num_epochs	=	50	
  all_scores	=	[]	
  # skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)
  kf = KFold(n_splits=k,shuffle=Kshuffle,random_state=Krandom_state)
  for train_index, test_index in kf.split(X_prime,y_prime):
    X_train, X_val = X_prime[train_index], X_prime[test_index]
    y_train, y_val = y_prime[train_index], y_prime[test_index]
    model.fit(X_train,y_train,epochs=num_epochs, batch_size=32,verbose=0)
    pred_values = model.predict(X_val)
    pred_values=np.round(pred_values)
    acc = accuracy_score(pred_values , y_val)
    all_scores.append(acc)
    
  avg_acc_score = sum(all_scores)/k
  print(Bold+"Traininng avg accuracy:"+FormatEnd,avg_acc_score)
  return model

## Model 1

In [ ]:
model1	=	Sequential()
input_size=X_prime.shape[1]
model1.add(Dense(input_size,	input_dim=input_size,	activation='relu'))	
model1.add(Dense(250,	activation='relu'))	
model1.add(Dense(100,	activation='relu'))	
model1.add(Dense(4,activation='softmax'))

#	2.	compile	the	network
model1.compile(loss='categorical_crossentropy',	optimizer='adam')

print(Bold+UnderLine+"Model1"+FormatEnd)
model=kfold_train(model=model1,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

Model1
Traininng avg accuracy: 0.7452297794117647


### Evaluate Model 1

In [ ]:
winePreds=	model1.predict(X_test)
winePredType =	np.argmax(winePreds,	axis=1)
wineActualType =	np.argmax(y_test,	axis=1)
correct	=	accuracy_score(wineActualType,	winePredType)
print(Bold+UnderLine+"Accuracy Prediction Model1:"+FormatEnd,correct)

Accuracy Prediction Model1: 0.6375


## Model 2

In [ ]:
#best model
model2	=	Sequential()
input_size=X_prime.shape[1]
print(input_size)
model2.add(Dense(input_size,	input_dim=input_size,	activation='relu'))	
model2.add(Dense(250,	activation='relu'))	
model2.add(Dense(100,	activation='relu'))	
model2.add(Dense(4,activation='sigmoid'))

#	2.	compile	the	network
model2.compile(loss='binary_crossentropy',	optimizer='adam',metrics=['accuracy'])

print(Bold+UnderLine+"Model2"+FormatEnd)
model2=kfold_train(model=model2,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

11
Model2
Traininng avg accuracy: 0.6725061274509804


### Evaluate Model2

In [ ]:
winePreds=	model2.predict(X_test)
winePredType =	np.argmax(winePreds,	axis=1)
wineActualType =	np.argmax(y_test,	axis=1)
correct	=	accuracy_score(wineActualType,	winePredType)
print(Bold+UnderLine+"Accuracy Prediction Model2:"+FormatEnd,correct)

Accuracy Prediction Model2: 0.634375


## Model 3


In [ ]:
model3	=	Sequential()
input_size=X_prime.shape[1]
print(input_size)
model3.add(Dense(input_size,	input_dim=input_size))	
model3.add(Dense(input_size**2,	activation='tanh'))	
model3.add(Dense(input_size,	activation='tanh'))
model3.add(Dense(4,activation='sigmoid',use_bias=True))

#	2.	compile	the	network
model3.compile(loss='binary_crossentropy',	optimizer='adam',metrics=['accuracy'])

print(Bold+UnderLine+"Model3"+FormatEnd)
model3=kfold_train(model=model3,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

11
Model3
Traininng avg accuracy: 0.5520618872549019


### Evaluate

In [ ]:
winePreds=	model3.predict(X_test)
winePredType =	np.argmax(winePreds,	axis=1)
wineActualType =	np.argmax(y_test,	axis=1)
correct	=	accuracy_score(wineActualType,	winePredType)
print(Bold+UnderLine+"Accuracy Prediction Model3:"+FormatEnd,correct)

Accuracy Prediction Model3: 0.671875


# PART B


##Parse Data

In [ ]:
saratogaFile='/content/drive/Shareddrives/Team 8- Neural Network/Assignment-8/SaratogaHomes.csv'
df = pd.read_csv(saratogaFile, header=0)
df.head()

,Price,Lot.Size,Waterfront,Age,Land.Value,New.Construct,Central.Air,Fuel.Type,Heat.Type,Sewer.Type,Living.Area,Pct.College,Bedrooms,Fireplaces,Bathrooms,Rooms
0,132500,0.09,0,42,50000,0,0,3,4,2,906,35,2,1,1.0,5
1,181115,0.92,0,0,22300,0,0,2,3,2,1953,51,3,0,2.5,6
2,109000,0.19,0,133,7300,0,0,2,3,3,1944,51,4,1,1.0,8
3,155000,0.41,0,13,18700,0,0,2,2,2,1944,51,3,1,1.5,5
4,86060,0.11,0,0,15000,1,1,2,2,3,840,51,2,0,1.0,3


In [ ]:
num_col=len(df.columns)
num_row=len(df)
num_feat=num_col-1
X=df.iloc[:num_row, 1:num_col].values
X=zscore(X)   #normalize
y=df['Price']
y=y.values
y=zscore(y)


## Split data

In [ ]:
#split test and train data
#X prime and Yprime are the training data after split
X_prime, X_test, y_prime, y_test = train_test_split(X, y, test_size=0.20, random_state=None)
input_size=X_prime.shape[1]


## Kfold and Training

In [ ]:
def kfold_train(model,k,num_epochs,X_prime,y_prime,Kshuffle,Krandom_state):
  """
  Does Kfold and train the model
  Shows the avg accuracy
  returns the model after training
  """
  # k	=	10
  # num_epochs	=	50	
  all_scores	=	[]	
  # skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)
  kf = KFold(n_splits=k,shuffle=Kshuffle,random_state=Krandom_state)
  for train_index, test_index in kf.split(X_prime,y_prime):
    X_train, X_val = X_prime[train_index], X_prime[test_index]
    y_train, y_val = y_prime[train_index], y_prime[test_index]
    model.fit(X_train,y_train,epochs=num_epochs, batch_size=32,verbose=0)
    pred_values = model.predict(X_val)
    acc = r2_score(y_val,pred_values)
    all_scores.append(acc)
    
  avg_acc_score = sum(all_scores)/k
  print(Bold+"Traininng avg accuracy:"+FormatEnd,avg_acc_score)
  return model

      

## Model 1

In [ ]:
#	1.	create	a	NN	model
model1	=	Sequential()
model1.add(Dense(64,	input_dim=input_size,	activation='relu'))	
model1.add(Dense(64,	activation='relu'))	
model1.add(Dense(1))
#	2.	compile	the	network and	print	a	summary
model1.compile(optimizer='rmsprop',	loss='mse',	metrics=['mae'])
input_size
#	3.	fit	the	network
print(Bold+UnderLine+"Model1"+FormatEnd)
model1=kfold_train(model=model1,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

Model1
Traininng avg accuracy: 0.7697215084825673


### Evaluate Model 1


In [ ]:
saratogaPreds=	model1.predict(X_test)
correct=r2_score(y_test,saratogaPreds)
print(Bold+UnderLine+"Accuracy Prediction Model1:"+FormatEnd,correct)

Accuracy Prediction Model1: 0.25669049529918875


## Model 2

In [ ]:
#	1.	create	a	NN	model
model2	=	Sequential()
model2.add(Dense(input_size,	input_dim=input_size,	activation='tanh'))	
model2.add(Dense(input_size**2,	activation='tanh'))	
model2.add(Dense(1))
#	2.	compile	the	network and	print	a	summary
model2.compile(optimizer='rmsprop',	loss='mse',	metrics=['mae'])
input_size
#	3.	fit	the	network
print(Bold+UnderLine+"Model2"+FormatEnd)
model1=kfold_train(model=model2,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

Model2
Traininng avg accuracy: 0.6715843927975784


### Evaluate Model 2

In [ ]:
saratogaPreds=	model2.predict(X_test)
correct=r2_score(y_test,saratogaPreds)
print(Bold+UnderLine+"Accuracy Prediction Model1:"+FormatEnd,correct)


Accuracy Prediction Model1: 0.5963764570059724


## Model 3

In [ ]:
#	1.	create	a	NN	model
model3	=	Sequential()
model3.add(Dense(input_size,	input_dim=input_size))	
model3.add(Dense(input_size**3,	activation='tanh'))	
model3.add(Dense(input_size**2,	activation='tanh'))	
model3.add(Dense(input_size,	activation='tanh'))	
model3.add(Dense(1,use_bias = True))
#	2.	compile	the	network and	print	a	summary
model3.compile(optimizer='SGD', loss='mse', metrics=['mse','accuracy'])
input_size
#	3.	fit	the	network
print(Bold+UnderLine+"Model3"+FormatEnd)
model1=kfold_train(model=model3,k=5,num_epochs=50,X_prime=X_prime,y_prime=y_prime,Kshuffle=True,Krandom_state=1)

Model3
Traininng avg accuracy: 0.6750798072112179


### Evaluate Model 3

In [ ]:
saratogaPreds=	model3.predict(X_test)
correct=r2_score(y_test,saratogaPreds)
print(Bold+"Accuracy Prediction Model1:"+FormatEnd,correct)


Accuracy Prediction Model1: 0.5591429137053947
